# Enzyme Stability Predition on Kaggle with ChatGPT

https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/

https://chat.openai.com/chat

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# read the train.csv file into a dataframe called "train_df"
train_df = pd.read_csv("train.csv")

# read the test.csv file into a dataframe called "test_df"
test_df = pd.read_csv("test.csv")

In [ ]:
# print the first 5 rows of the train_df dataframe
print(train_df.head())

# print the first 5 rows of the test_df dataframe
print(test_df.head())

# print the datatypes of each column in the train_df dataframe
print(train_df.dtypes)

# print the datatypes of each column in the test_df dataframe
print(test_df.dtypes)

In [ ]:
# calculate the percentage of rows with null or NaN values
train_null_percentage = train_df.isnull().sum() / train_df.shape[0] * 100
test_null_percentage = test_df.isnull().sum() / test_df.shape[0] * 100

# print the result
print(f"Train null percentage: {train_null_percentage}")
print(f"Test null percentage: {test_null_percentage}")

In [ ]:
# remove any rows that contain NaN or null values
train_df_filtered = train_df.dropna()

In [ ]:
# Write train_df_filtered to a new csv file called "train_filtered.csv"
train_df_filtered.to_csv("train_filtered.csv", index=False)

In [ ]:
# Reduce the number of rows in the dataframe
# TODO: remove this line before submitting for final training
train_df_filtered = train_df_filtered.head(6)
test_df = test_df.head(5)

# EDA and Normalization

In [ ]:
# Join the train and test dataframes for normalization
combined_df = pd.concat([train_df_filtered, test_df])

In [ ]:
# create a histogram of the "pH" column
combined_df["pH"].plot.hist()

# add a title and x-axis label
plt.title("pH")
plt.xlabel("pH")

# show the plot
plt.show()

In [ ]:
# create a histogram of the "tm" column
train_df_filtered["tm"].plot.hist()

# add a title and x-axis label
plt.title("tm")
plt.xlabel("tm")

# show the plot
plt.show()

In [ ]:
# Normalize the "pH" column
ph_mean = combined_df["pH"].mean()
ph_std = combined_df["pH"].std()
test_df["pH"] = (test_df["pH"] - ph_mean) / ph_std
train_df_filtered["pH"] = (train_df_filtered["pH"] - ph_mean) / ph_std

# Write test_df to a new csv file called "test_normalized.csv"
test_df.to_csv("test_normalized.csv", index=False)

# Normalize the "tm" column
tm_mean = train_df_filtered["tm"].mean()
tm_std = train_df_filtered["tm"].std()
train_df_filtered["tm"] = (train_df_filtered["tm"] - tm_mean) / tm_std

# Print tm_mean and tm_std
print(f"tm_mean: {tm_mean}")
print(f"tm_std: {tm_std}")


# Write train_df_filtered to a new csv file called "train_normalized.csv"
train_df_filtered.to_csv("train_normalized.csv", index=False)

In [ ]:
# create a histogram of the "tm" column
train_df_filtered["tm"].plot.hist()

# add a title and x-axis label
plt.title("tm")
plt.xlabel("tm")

# show the plot
plt.show()

In [ ]:
# create a histogram of the "pH" column
train_df_filtered["pH"].plot.hist()

# add a title and x-axis label
plt.title("pH")
plt.xlabel("pH")

# show the plot
plt.show()

In [ ]:
# create a histogram of the "pH" column
test_df["pH"].plot.hist()

# add a title and x-axis label
plt.title("pH")
plt.xlabel("pH")

# show the plot
plt.show()

# DataSource to Category

In [ ]:
# find all the unique values in the "data_source" column
data_sources = combined_df["data_source"].unique()

# print the result
print(data_sources)

# print the length of the result
print(len(data_sources))

In [ ]:
# Create a one-hot encoding of the "data_source" column
one_hot_encoding = pd.get_dummies(df["data_source"])

# Join the one-hot encoding with the original dataframe
df_with_one_hot = df.join(one_hot_encoding)

# Embedding of Data Source

In [ ]:
import gc
import torch
from transformers import BertTokenizer, BertForPreTraining

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForPreTraining.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")
model.eval()

# Clear out any stale data in the GPU
torch.cuda.empty_cache()
# gc.collect()

# Encode  the "data_source" column in the dataframe
with torch.no_grad():
    train_df_filtered["data_source_encoded_BERT"] = train_df_filtered["data_source"].apply(lambda x: model(**tokenizer(x, return_tensors="pt")).prediction_logits.cpu().numpy().flatten())
    test_df["data_source_encoded_BERT"] = test_df["data_source"].apply(lambda x: model(**tokenizer(x, return_tensors="pt")).prediction_logits.cpu().numpy().flatten())

In [ ]:
train_df_filtered["data_source_encoded_BERT"].head()

## Embedding of Sequence

In [ ]:
import torch
import esm

# Clear out any stale data in the GPU
torch.cuda.empty_cache()

# Load the ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
# Big Boy model
# model, alphaber = esm.pretrained.esm2_t48_15B_UR50D()
model.eval()

def encode_sequence(sequence):
    """Encode a sequence using the ESM-2 model."""

    # Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
    batch_converter = alphabet.get_batch_converter()
    batch_labels, batch_strs, batch_tokens = batch_converter([('0', sequence)])
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)    
    token_representations = results["representations"][33]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

    encoded_sequence = sequence_representations[0].cpu().numpy()

    print(f"Converted {sequence} into {encoded_sequence}")
    return encoded_sequence

# encode the "protein_sequence" column in the train_df dataframe into a new column
train_df_filtered["esm2_t33_650M_UR50D_sequence"] = train_df_filtered["protein_sequence"].apply(encode_sequence)
test_df["esm2_t33_650M_UR50D_sequence"] = test_df["protein_sequence"].apply(encode_sequence)


In [ ]:
# Define the input and output columns
col = "esm2_t33_650M_UR50D_sequence"

# Find the length of the arrays in the column
array_length = train_df_filtered[col].apply(len).max()

# Generate the output column names
esm2_output_cols = ["esm2_t33_650M_UR50D_sequence_{}".format(i) for i in range(array_length)]

# Convert the column to multiple columns containing single numbers
train_df_filtered[esm2_output_cols] = pd.DataFrame(train_df_filtered[col].tolist(), index=train_df_filtered.index)
test_df[esm2_output_cols] = pd.DataFrame(test_df[col].tolist(), index=test_df.index)


print(train_df_filtered[esm2_output_cols].head())
print(test_df[esm2_output_cols].head())

In [ ]:
# Define the input and output columns
col = "data_source_encoded_BERT"

# Find the length of the arrays in the column
array_length = train_df_filtered[col].apply(len).max()

# Generate the output column names
bert_output_cols = ["data_source_encoded_BERT_{}".format(i) for i in range(array_length)]

# Convert the column to multiple columns containing single numbers
train_df_filtered[bert_output_cols] = pd.DataFrame(train_df_filtered[col].tolist(), index=train_df_filtered.index)
test_df[bert_output_cols] = pd.DataFrame(test_df[col].tolist(), index=test_df.index)

print(train_df_filtered[bert_output_cols].head())
print(test_df[bert_output_cols].head())

In [ ]:
from sklearn.linear_model import LinearRegression

# Define the input and target columns
# X = train_df_filtered[["esm2_t33_650M_UR50D_sequence", "data_source_encoded_BERT", "pH"]]
# X = train_df_filtered[bert_output_cols + esm2_output_cols + ["pH"]]
# X = train_df_filtered[bert_output_cols + esm2_output_cols]
X = train_df_filtered[esm2_output_cols]
y = train_df_filtered["tm"]

# Train the regression model
reg = LinearRegression().fit(X, y)

In [ ]:
# Define the input columns
# X = test_df[["esm2_t33_650M_UR50D_sequence", "data_source_encoded_BERT", "pH"]]
X = test_df[esm2_output_cols]

# Use the trained "reg" model to make predictions on the "tm" column
tm_predictions = reg.predict(X)

# Un-normalize the predictions
tm_predictions = tm_predictions * tm_std + tm_mean

# Add the predictions to a new column in the dataframe
test_df["tm"] = tm_predictions

print(test_df[['seq_id', 'tm']].head())

# Write the dataframe to the output file
test_df[['seq_id', 'tm']].to_csv("submission.csv", index=False)

# Making Train Ready CSV files

In [ ]:
for dataset in [
    'train_filtered',
    'test',
]:
    for embedding_model in [
        'embeddings_esm2_t33_650M_UR50D',
        # 'embeddings_esm1v_t33_650M_UR90S_1',
    ]:
        # Read in the two CSV files
        df1 = pd.read_csv(f"{dataset}_{embedding_model}.csv")
        df2 = pd.read_csv(f"{dataset}_normalized.csv")

        # Join the two dataframes on the "seq_id" column
        df_merged = df1.merge(df2, on="seq_id")

        # Get only the columns we will use for training
        if dataset == 'test':
            df_subset = df_merged[["seq_id", "pH"] + [f"latent_{i}" for i in range(1280)]]
        else:
            df_subset = df_merged[["seq_id", "tm", "pH"] + [f"latent_{i}" for i in range(1280)]]

        # Print the merged dataframe
        print(df_subset.head(5))

        # Write the df_subset dataframe to a CSV file called "train_ready_embeddings_esm2_t33_650M_UR50D.csv"
        df_subset.to_csv(f"{dataset}_ready_{embedding_model}.csv", index=False)